## Week4
#### 그냥 인터넷에서 본 이것저것

tensor 2 부터 지원되는 tf.gradients는 graph로 표현된 층으로부터의 미분값들을 구한다.<br>
궁금해서 tf.graph를 찾아봤는데, 이는 파이썬 내에서만의 operation으로 구성된 기존 레이어들에서 벗어나,
graph 형식의 data structure를 구현하여 파이썬 interpreter가 없는 embeded, mobile, server 등에서 tf를 활용할 수 있도록 한다고 한다.

### 하긴
현재 보고 있는 강의도 그렇다. 기본적인 원리를 설명하기 위해 numpy로 직접 하나하나 구현하는 operation이지만, 이것은 딥러닝 프레임워크가 나옴에 따라 구조화 되어 우리가 편히 사용하는 것이니까. <br>
우선적으로 원리를 알기 위해 이 강의를 보고 있음을 잊지 말자.
----
hidden layer의 weight를 그림으로 나타내면, 서로 다른 빗면으로 나타나는 그림 여러장을 구할 수 있는데, 이는 DNN에서 주어진 iuput image에서 각 빗면이 어디에 위치하는지를 구하는 것이라고 한다.(아마 특이한 케이스일 것이다. 이미 다 세팅된..)<br>

그리고 layer를 지날때마다 x의 사이즈가 줄어들면서 눈, 코, 입 단위의 프레임이 턱, 이마, 안면 등으로 확대되며 학습이 진행된다. <br>
즉, 작은 부분을 찾아서 그들을 합쳐가며 복잡한 부분을 찾을 수 있는 것이라고..

마지막에, 작은 모델부터 시작해서 (logistic regression) 문제를 해결할 수 있는 모델을 찾는 것이 효율적이라고 하셨다.<br>
(물론, 문제에 따라 접근해야겟지만..) 이런 응용력이 기반되어야 할듯하다.

**loss가 구해짐에 따라**

backpropagation이 가능해지며 역으로 전파하고, 현재 loss 공식에 맞는 derivative term이 구해지고 전파되며 derivative weight가 구해짐

어쨌든.. 오늘은 DNN의 backpropagation을 배웠음.

In [1]:
import tensorflow.keras.layers as Layer
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [12]:
import os
import pickle

x = open('/root/storage/DATA/pickle/checked','rb')
data = pickle.load(x)
x.close()

cycle = np.random.permutation(25)
exp_data = data[cycle[:10]].reshape(-1,data.shape[1],data.shape[-1]*data.shape[-2])

In [22]:
def build(shape):
    x_input = Layer.Input(shape)
    
    x = Layer.Dense(20,activation='tanh')(x_input)
    x = Layer.Dense(50,activation='tanh')(x)
    x = Layer.Dense(150,activation='tanh')(x)
    
    model = Model(inputs=x_input,outputs=x,name='custom')
    return model
#x = np.random.randn(7,7760)

In [23]:
exp_shape = exp_data[0,0].shape
print(exp_shape)
model = build(exp_shape)

(7760,)


In [24]:
model.summary()

Model: "custom"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 7760)]            0         
_________________________________________________________________
dense_18 (Dense)             (None, 20)                155220    
_________________________________________________________________
dense_19 (Dense)             (None, 50)                1050      
_________________________________________________________________
dense_20 (Dense)             (None, 150)               7650      
Total params: 163,920
Trainable params: 163,920
Non-trainable params: 0
_________________________________________________________________


In [25]:
model(exp_data[:50])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(10, 200, 150), dtype=float32, numpy=
array([[[-0.44710606, -0.03479009,  0.2776845 , ...,  0.03071656,
          0.04172621, -0.2214867 ],
        [ 0.36780643, -0.03157755,  0.20750643, ...,  0.15242286,
         -0.58608115, -0.59799516],
        [ 0.01328856,  0.40988535, -0.5281662 , ...,  0.06496996,
          0.04788193, -0.27807432],
        ...,
        [ 0.0117518 ,  0.40927723, -0.27528048, ..., -0.08732712,
         -0.11722292, -0.34099525],
        [ 0.20032226, -0.37790754, -0.43276054, ..., -0.5427054 ,
          0.3727481 ,  0.00213731],
        [-0.00595208,  0.6486525 , -0.0945396 , ...,  0.0463046 ,
          0.16105889, -0.12292836]],

       [[-0.21661524,  0.22649434, -0.7352831 , ..., -0.24333984,
          0.18397786,  0.26458293],
        [-0.2824777 ,  0.40678188, -0.05302149, ...,  0.20446025,
         -0.15899362,  0.3583934 ],
        [ 0.03856886,  0.20995648, -0.41758028, ...,  0.1178822 ,
          0.15441886,  0.3110795 ],
        ...

## 일단 너무 추우니까 철수 ;